In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from random import shuffle
import random
import math

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
def efficient_frontier(returns, n_samples=50, gamma_low=-1, gamma_high=10):
    """
    construye un conjunto de problemas de programación cuádrática
    para inferir la frontera eficiente de Markovitz. 
    En cada problema el parámetro gamma se cambia para aumentar
    la penalización del riesgo en la función de maximización.
    """
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  w >= 0]) 
    # Equivalente 
    #prob = cp.Problem(cp.Minimize(risk - gamma*ret), 
    #                  [cp.sum(w) == 1,  w >= 0])   
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []    
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)   
    return ret_data, risk_data, gamma_vals, portfolio_weights


def optimal_portfolio(returns):
    """
    A partir del calculo de la frontera eficiente de Markowitz
    devuelve el portfolio optimo: su ratio de sharpe, rentabilidad,
    riesgo y pesos.
    """
    ret_data, risk_data, _, portfolio_weights = efficient_frontier(returns)

    sharpes = ret_data/risk_data 
    idx = np.argmax(sharpes)
    optimal_ret, optimal_risk = ret_data[idx], risk_data[idx]
    optimal_portfolio = pd.Series(portfolio_weights[idx],
                                index=returns.columns).abs().round(3)

    optimal_portfolio = optimal_portfolio.div(optimal_portfolio.sum()).round(3)
                                
    return sharpes[idx], optimal_ret, optimal_risk, optimal_portfolio

In [ ]:
funds = pd.read_csv('funds.csv', index_col=0, parse_dates=[0])
funds

In [ ]:
returns = np.log(funds).diff().dropna()
returns

# ALGORTIMO GENETICO

 Generar aleatoriamente una población inicial

 Repetir hasta que se cumpla la condición de parad

     Evaluar los individuos según la función objetivo (maximizar beneficio, minimizar distancia)

     Seleccionar padres (en función de la evaluación anterior).
    
     Cruzamiento: recombinar material genético de los padres

     Evaluar si el individuo, o parte de su código genético muta

     Reemplazo generacional
    
     Generación = Generación +1

 Cuando se cumpla condición de parada, devolver mejor solución.

In [ ]:
def evalua_cromosomas(list_info_cromo):
    """
    Evalua los cromosomas de una población.
    Normaliza sharpe para la seleccion de padres posterior, 
    y devuelve el cromosoma con mayor sharpe.
    Devuelve una lista de tuplas (max_sharpe, list_info_cromo)
    """
    # Lista con los sharpes
    sharpes = [item[0] for item in list_info_cromo]
    # Elimino sharpes infinitos
    sharpes = [np.nanmin(sharpes) if item == np.inf else item for item in sharpes]
    
    # Normalizo sharpe con el minimo y maximo
    norm_sharpes = ((sharpes - np.nanmin(sharpes))/(np.nanmax(sharpes) - np.nanmin(sharpes))).tolist()
    
    # Agrego la lista de norm_sharpes a la lista de cromosomas con el resto de info
    list_info_cromo = [[norm_sharpe, list_info[0], list_info[1], list_info[2], list_info[3]] for norm_sharpe, list_info in zip(norm_sharpes, list_info_cromo)]

    max_sharpe = []
    [max_sharpe.append(item) if item[0] == 1 else '' for item in list_info_cromo]
    max_sharpe = max_sharpe[0][1:]
        
    return max_sharpe, list_info_cromo


In [ ]:
def nueva_generacion(cromo_max_sharpe, list_info_cromo):
    """
    Creación de nueva generacion.

    Obtencion de padres por ruleta, 2 padres generan 1 hijo.

    Mutacion del hijo, se incluye una probabilidad de mutacion y se 
    añaden entre 1 y 4 nuevos activos, escogidos aleatoriamente.
    
    Reemplazo total, la generación de hijos reemplaza a sus padres, 
    a excepción del mejor de la generación anterior.
    """

    list_info_hijos = []

    for crom in range(n_cromosomas - 1):
        
        # Reordeno la lista de cromosomas aleatoriamente
        shuffle(list_info_cromo)
        # Obtengo una lista con probabilidades de seleccionar a un padre
        rand_probs = np.random.uniform(0, 1, size=len(list_info_cromo))
        # A los padres les asigno un 1 si se seleccionaron, 0 si no
        padres = [1 if rand_prob < list_info[0] else 0 for rand_prob, list_info in zip(rand_probs, list_info_cromo)]

        # Ahora voy a seleccionar los padres que sean 1
        new_padres = []
        [new_padres.append(list_info) if padre == 1 else '' for padre, list_info in zip(padres, list_info_cromo)]
        # Me quedo con los 2 primeros padres
        padres = new_padres[:2]


        # cojo los activos con peso mayor que 0 de cada padre
        activos_padres = padres[1][4][padres[1][4] > 0].index.to_list() + padres[0][4][padres[0][4] > 0].index.to_list()
        # get unique elements from list
        activos_padres = list(set(activos_padres))

        shuffle(activos_padres)
        activos_padres = activos_padres[:min(len(activos_padres), random.randint(1,16))]

        # # Calculo correlaciones entre los activos de los padres
        # corr_padres = returns.loc[:, activos_padres].corr().dropna(axis=0, how='all').dropna(axis=1, how='all')
        # corr_padres = corr_padres[corr_padres != 1]
        # print(corr_padres)

        # Mutaciones
        activos_mutacion = returns.iloc[:, random.sample(range(0,n_genes), random.randint(0,4))].columns.to_list()
        activos_hijo = activos_padres + activos_mutacion
        activos_hijo = list(set(activos_hijo))


        # Hijo
        returns_hijo = returns.loc[:, activos_hijo]
    
        sharpe, ret, std, weights = optimal_portfolio(returns_hijo)
        list_info_hijos.append([sharpe, ret, std, weights])
        # print(len(weights), len(weights[weights > 0]))


    # Añado el padre con mayor sharpe a la lista de hijos
    list_info_hijos.append(cromo_max_sharpe)

    return list_info_hijos

### Generacion aleatoria inicial

In [ ]:
# genes = activos
n_genes = returns.shape[1]

# cromosomas = inversores
n_cromosomas = round(n_genes * (50/(5+n_genes)))

# n_cromosomas = 50

print(n_genes, n_cromosomas)

In [ ]:
max_sharpe = [0]

while max_sharpe[0] <= 1:

    list_info_cromo = []

    # Para cada cromosoma de la población inicial escogo aleatoriamente una cantidad de activos
    # entre 1 y 20, y calculo la cartera que maximiza el sharpe. 
    for i in range(0, n_cromosomas):
        returns_random = returns.iloc[:, random.sample(range(0,n_genes), random.randint(1,20))]
        sharpe, ret, std, weights = optimal_portfolio(returns_random)
        list_info_cromo.append([sharpe, ret, std, weights])

    # Obtengo el cromosoma con mayor sharpe, y la lista de cromosomas con sharpe normalizado
    max_sharpe, list_info_cromo = evalua_cromosomas(list_info_cromo)

print(f'Generacion 0 - sharpe {round(max_sharpe[0], 4)}')


# Lista de listas con los activos que componen la cartera de mejor sharpe de cada generacion,
# para luego poder sacar la evolucion de la frontera eficiente por generaciones
list_activos_mejor_frontera = []

list_activos_mejor_frontera.append(max_sharpe[3].index.to_list())

# Lista con los sharpes maximos de cada generacion, se usara para la condicion de parada
max_sharpes = []

max_sharpes.append(max_sharpe[0])

last_sharpe = 0.01
diff_sharpes = [0.01]

idx = 1

# Bucle de las sucesivas generaciones
while not all(i < 0.0001 for i in diff_sharpes[-30:]):
    
    list_info_cromo = nueva_generacion(max_sharpe, list_info_cromo)

    max_sharpe, list_info_cromo = evalua_cromosomas(list_info_cromo)


    max_sharpes.append(max_sharpe[0])

    list_activos_mejor_frontera.append(max_sharpe[3].index.to_list())

    diff_sharpes.append((max_sharpe[0] - last_sharpe)/last_sharpe)

    last_sharpe = max_sharpe[0]

    print(f'Generacion {idx} - sharpe {round(max_sharpe[0], 4)}')
    idx += 1
    print(diff_sharpes[-30:], not all(i < 0.0001 for i in diff_sharpes[-30:]))


In [ ]:
plt.plot(max_sharpes)

In [ ]:
max_sharpe[3]

In [ ]:
max_sharpe[3][max_sharpe[3] > 0]

In [ ]:
for i in range(0, len(list_activos_mejor_frontera)):
    if (i % 5 == 0) | (i == len(list_activos_mejor_frontera)-1):
        print(f'Generacion {i}')

In [ ]:
# Plot de la evolucion de la mejor frontera eficiente de cada generacion
list_rets_risks = []
list_optimal_rets_risks = []
for activos, i in zip(list_activos_mejor_frontera, range(0, len(list_activos_mejor_frontera))):
    if i % 5 == 0:
        ret, risk, _, _ = efficient_frontier(returns.loc[:, activos])
        list_rets_risks.append([ret, risk])

        _, optimal_ret, optimal_risk, _ = optimal_portfolio(returns.loc[:, activos])
        list_optimal_rets_risks.append([optimal_ret, optimal_risk])


fig, ax = plt.subplots(figsize=(20, 10))

idx = 0

for lista, optimals in zip(list_rets_risks, list_optimal_rets_risks):
    ax.plot(lista[1], lista[0], 'y', label=f'Generacion {idx}')
    ax.plot(optimals[1], optimals[0], '*', markersize=15)
    idx += 5


colormap = plt.cm.jet #nipy_spectral, Set1,Paired   
colors = [colormap(i) for i in np.linspace(0, 0.95,len(ax.lines))]
for i,j in enumerate(ax.lines):
    if i % 2 == 0:
        j.set_color(colors[i])
    else:
        j.set_color(colors[i-1])

ax.set_xlim(right=list_rets_risks[-1][1].max())
ax.set_ylim(top=list_rets_risks[-1][0].max())
plt.legend()